# Cantonese Translation Project

## Installing Dependencies Required

In [1]:
!pip3 install transformers
!pip3 install transformers[sentencepiece]
!pip3 install sacremoses
!pip3 install datasets
!pip3 install transformers[torch]
!pip3 install sacrebleu
!pip3 install evaluate
!pip install chinese-converter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.1 MB/s eta 0:00:00


## Imports

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import Seq2SeqTrainer
from transformers import BertTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
import random
from tqdm import tqdm
import gc
import torch
import evaluate
import numpy as np
import chinese_converter

## Import Data

In [3]:
# Folder and File Name
base_path = "/content/drive/MyDrive/Cantonese-NLP/Processed-Data"
folders = ["/train-short","/train","/dev","/test"]
file_name = ["/yue.txt","/en.txt"]

In [4]:
yue = []
en = []
for folder in folders:
  with open(base_path+folder+file_name[0],'r') as f_yue, open(base_path+folder+file_name[1],'r') as f_en:
    yue.append(f_yue.read().splitlines())
    en.append(f_en.read().splitlines())


In [5]:
# Concatenate Short and Long Training Example
yue[1] = yue[0] + yue[1]
del yue[0]
en[1] = en[0] + en[1]
del en[0]

In [6]:
import pandas as pd

train = zip(yue[0], en[0])
dev = zip(yue[1], en[1])
test = zip(yue[2], en[2])

train_df = pd.DataFrame(train, columns = ['zh','en'])
dev_df = pd.DataFrame(dev, columns = ['zh','en'])
test_df = pd.DataFrame(test, columns = ['zh','en'])

In [7]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset


dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'dev': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)})


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['zh', 'en'],
        num_rows: 38044
    })
    dev: Dataset({
        features: ['zh', 'en'],
        num_rows: 3001
    })
    test: Dataset({
        features: ['zh', 'en'],
        num_rows: 3000
    })
})

## OPUS-MT Model

Load Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq, Seq2SeqTrainer

model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Testing Model without finetuning

In [ ]:
import random
n = random.randint(0, len(dataset["test"]) - 1)
test = dataset["test"][n]
test_string = test["zh"]
print("Orginal Data: " + test_string)
tokens = tokenizer(test_string, return_tensors="pt", padding=False).to("cuda")
translated = model.generate(**tokens)
print("Model Translation: " + tokenizer.batch_decode(translated, skip_special_tokens=True)[0])
print("Dataset Translation: " + test["en"])

Orginal Data: 三叔份人好疏爽㗎！如果你經濟上真係出咗問題嘅，你可以試下揾佢幫幫手㗎噃。
Model Translation: Uncle San is so lazy! If there's an economic problem, you can try to help him.
Dataset Translation: Uncle Three is very generous, you should seek his help if you are hit with a financial downturn.


In [ ]:
import evaluate
from tqdm import tqdm

metric = evaluate.load("sacrebleu")

source_lang = "zh"
target_lang = "en"


def compute_bleu(src_sentences, tgt_sentences, batch_size=32):
  pred = []

  for i in tqdm(range(0, len(src_sentences), batch_size)):
    batch = src_sentences[i:i + batch_size]
    encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
    translated_output = model.generate(**encoded_input)
    translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_output]
    pred.extend(translated_batch)

  res = metric.compute(predictions=pred, references=tgt_sentences)
  return res["score"]


bleu_score = compute_bleu(dataset["test"][source_lang], dataset["test"][target_lang])
print(f"Bleu Score Before Finetuning with {model_name}: {bleu_score}")

100%|██████████| 94/94 [02:43<00:00,  1.74s/it]


Bleu Score Before Finetunign with Helsinki-NLP/opus-mt-zh-en: 10.48731370511051


### Preprocessing Data

In [ ]:
max_length = 128
source_lang = "zh"
target_lang = "en"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples]
    targets = [ex[target_lang] for ex in examples]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/38044 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 38044
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
    dev: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3001
    })
})

### Fine-Tuning Datasets

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
model_name = model_name.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.376200,2.290523,14.753000,16.814700
2,1.071100,2.265406,15.579100,16.428200
3,1.036700,2.352300,15.633000,16.642500
4,0.768900,2.423655,15.566900,16.797700
5,0.569700,2.512335,16.284000,16.423900
6,0.438900,2.602198,16.199600,16.230300
7,0.329500,2.674185,16.415500,16.426200
8,0.258100,2.744746,16.650600,16.419500
9,0.200700,2.798014,16.433200,16.382200
10,0.164400,2.816994,16.604800,16.409200


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=11890, training_loss=0.6239168252134845, metrics={'train_runtime': 2277.365, 'train_samples_per_second': 167.053, 'train_steps_per_second': 5.221, 'total_flos': 2698951798554624.0, 'train_loss': 0.6239168252134845, 'epoch': 10.0})

In [ ]:
trainer.evaluate(tokenized_dataset["test"])

In [ ]:
trainer.save_model("/content/drive/MyDrive/Cantonese-NLP/model/opus-mt-zh-en-finetuned")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Cantonese-NLP/model/opus-mt-zh-en-finetuned", local_files_only=True).to("cuda")

In [ ]:
bleu_score = compute_bleu(dataset["test"][source_lang], dataset["test"][target_lang])
print(f"Bleu Score After Finetuning with {model_name}: {bleu_score}")

100%|██████████| 94/94 [02:28<00:00,  1.58s/it]


Bleu Score Before Finetuning with Helsinki-NLP/opus-mt-zh-en: 15.57886606811519


In [ ]:
test_string = test["zh"]
print("Original Data: " + test_string)
tokens = tokenizer(test_string, return_tensors="pt", padding=False).to("cuda")
translated = model.generate(**tokens)
print("Model Translation: " + str([tokenizer.decode(t, skip_special_tokens=True) for t in translated]))
print("Original Translation: " + test["en"])

Original Data: 三叔份人好疏爽㗎！如果你經濟上真係出咗問題嘅，你可以試下揾佢幫幫手㗎噃。
Model Translation: ['Uncle Sam is very absent-minded! If you have serious problems with your economy, you can ask him for help.']
Original Translation: Uncle Three is very generous, you should seek his help if you are hit with a financial downturn.


## NLLB-200

### Preprocessing Data

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq, Seq2SeqTrainer

model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="yue_Hant", tgt_lang="eng_Latn"
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to("cuda")

In [ ]:
# import random
# n = random.randint(0, len(dataset["test"]))
test = dataset["test"][n]
test_string = test["zh"]
print("Orginal Data: " + test_string)


inputs = tokenizer(test["zh"], return_tensors="pt", padding=True, truncation=True).to("cuda")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=128
)

print("Model Translation: " + tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])
print("Dataset Translation: " + test["en"])

Orginal Data: 三叔份人好疏爽㗎！如果你經濟上真係出咗問題嘅，你可以試下揾佢幫幫手㗎噃。
Model Translation: You're so distracted! If you're really in financial trouble, you can try to get him to help you.
Dataset Translation: Uncle Three is very generous, you should seek his help if you are hit with a financial downturn.


In [ ]:
from tqdm import tqdm
import evaluate

metric = evaluate.load("sacrebleu")

def compute_bleu_nllb(src_sentences, tgt_sentences, batch_size=32):
  pred = []

  for i in tqdm(range(0, len(src_sentences), batch_size)):
    batch = src_sentences[i:i + batch_size]
    encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to("cuda")
    translated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=128)
    translated_batch = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    pred.extend(translated_batch)

  res = metric.compute(predictions=pred, references=tgt_sentences)
  return res["score"]


In [ ]:
bleu_score = compute_bleu_nllb(dataset["test"]["zh"], dataset["test"]["en"])
print(f"Bleu Score Before Finetuning with {model_name}: {bleu_score}")

100%|██████████| 94/94 [01:34<00:00,  1.00s/it]


Bleu Score Before Finetuning with facebook/nllb-200-distilled-600M: 11.13838860020088


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "zh"
target_lang = "en"
def preprocess_function(examples):
    inputs =  examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, return_tensors="pt", max_length=max_input_length, padding=True, truncation=True)
    labels = tokenizer(targets, return_tensors="pt", max_length=max_input_length, padding=True, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/38044 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

In [ ]:
batch_size = 8
model_name = model_name.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16 = True,
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.547400,0.600572,15.842200,19.038000
2,0.414000,0.597614,16.607400,18.659700
3,0.306900,0.628150,16.814300,18.765300
4,0.223700,0.682231,16.458600,19.210000
5,0.164400,0.742678,16.496000,19.010000
6,0.115100,0.810697,16.303200,18.980000
7,0.082300,0.860636,16.395300,19.389000
8,0.057100,0.903166,16.670100,19.030000
9,0.040800,0.928514,16.885000,19.135300
10,0.032100,0.942336,16.949700,19.079300


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=47560, training_loss=0.208083332838182, metrics={'train_runtime': 28677.0782, 'train_samples_per_second': 13.266, 'train_steps_per_second': 1.658, 'total_flos': 4.774283812995072e+16, 'train_loss': 0.208083332838182, 'epoch': 10.0})

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/Cantonese-NLP/model/{model_name}-finetuned")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="yue_Hant", tgt_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(f"/content/drive/MyDrive/Cantonese-NLP/model/nllb-200-distilled-600M-finetuned", local_files_only=True).to("cuda")

In [ ]:
test = dataset["test"][n]
test_string = test["zh"]
print("Orginal Data: " + test_string)


inputs = tokenizer(test["zh"], return_tensors="pt").to("cuda")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
)

print("Model Translation: " + tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])
print("Dataset Translation: " + test["en"])

Orginal Data: 三叔份人好疏爽㗎！如果你經濟上真係出咗問題嘅，你可以試下揾佢幫幫手㗎噃。
Model Translation: Uncle Sam is a sloppy person! If you really have a financial problem, you can try to ask him to help.
Dataset Translation: Uncle Three is very generous, you should seek his help if you are hit with a financial downturn.


In [ ]:
bleu_score = compute_bleu_nllb(dataset["test"]["zh"], dataset["test"]["en"])
print(f"Bleu Score after Finetuning with nllb-200-distilled-600M: {bleu_score}")

100%|██████████| 94/94 [01:24<00:00,  1.11it/s]


Bleu Score after Finetuning with nllb-200-distilled-600M: 16.121964105036355


## mBART

In [9]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name).to("cuda")
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer.src_lang = "zh_CN"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [10]:
import chinese_converter
from tqdm import tqdm
import evaluate

metric = evaluate.load("sacrebleu")

def compute_bleu(src_sentences, tgt_sentences, batch_size=32):
  pred = []
  source_lang = "zh_CN"
  target_lang = "en_XX"

  for i in tqdm(range(0, len(src_sentences), batch_size)):
    batch = src_sentences[i:i + batch_size]
    encoded_input = tokenizer(
        batch,
        padding=True,
        truncation=True,
        max_length = 512,
        return_tensors="pt"
    ).to("cuda")
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]
    )
    translated_batch = tokenizer.batch_decode(
        generated_tokens,
        skip_special_tokens=True
    )
    for ind, sentence in enumerate(translated_batch):
      translated_batch[ind] = chinese_converter.to_traditional(sentence)
    pred.extend(translated_batch)

  res = metric.compute(predictions=pred, references=tgt_sentences)
  return res["score"]


In [11]:
bleu_score = compute_bleu(dataset["test"]["zh"], dataset["test"]["en"])
print(f"Bleu Score Before Finetuning with {model_name}: {bleu_score}")

100%|██████████| 94/94 [01:55<00:00,  1.23s/it]


Bleu Score Before Finetuning with facebook/mbart-large-50-many-to-many-mmt: 8.315683152180513


In [12]:
max_input_length = 512
source_lang = "zh"
target_lang = "en"

def preprocess_function(examples):
    inputs =  examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(
        inputs,
        return_tensors="pt",
        max_length=max_input_length,
        padding=True,
        truncation=True
    )
    tokenizer.src_lang = "en_XX"
    labels = tokenizer(
        targets,
        return_tensors="pt",
        max_length=max_input_length,
        padding=True,
        truncation=True
    )
    tokenizer.src_lang = "zh_CN"
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/38044 [00:00<?, ? examples/s]

Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['zh', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 38044
    })
    dev: Dataset({
        features: ['zh', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3001
    })
    test: Dataset({
        features: ['zh', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [15]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [16]:
from transformers import Seq2SeqTrainingArguments
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    auto_find_batch_size = True,
    predict_with_generate=True,
    load_best_model_at_end=True,
)

In [17]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    for ind, sentence in enumerate(decoded_preds):
      decoded_preds[ind] = chinese_converter.to_traditional(sentence)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(
        decoded_preds,
        decoded_labels
    )
    result = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [18]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.598600,0.577818,14.882000,19.429900
2,0.334200,0.560304,16.225000,19.434500
3,0.138300,0.660802,16.207200,19.385900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=14268, training_loss=0.3802789137882781, metrics={'train_runtime': 3624.255, 'train_samples_per_second': 31.491, 'train_steps_per_second': 3.937, 'total_flos': 1.4145236936097792e+16, 'train_loss': 0.3802789137882781, 'epoch': 3.0})

In [20]:
trainer.save_model(f"{base_path}model/mBart-baseline")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


In [21]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(
    f"{base_path}model/mBart-baseline",
    local_files_only = True,
).to("cuda")
tokenizer.src_lang = "zh_CN"

In [22]:
bleu_score = compute_bleu(dataset["test"]["zh"], dataset["test"]["en"])
print(f"Bleu Score Before Finetuning with {model_name}: {bleu_score}")

100%|██████████| 94/94 [01:59<00:00,  1.27s/it]


Bleu Score Before Finetuning with facebook/mbart-large-50-many-to-many-mmt: 15.751320872414007


In [23]:
from google.colab import runtime
runtime.unassign()

## Model Evaluation

In [ ]:
with open("/content/drive/MyDrive/Cantonese-NLP/SoTA-Translations/bing_translated.txt") as f:
  bing_translated = f.read().splitlines()

In [ ]:
len(dataset["test"]["en"])

3000

In [ ]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")

res = sacrebleu.compute(predictions=bing_translated, references=dataset["test"]["en"])
print(res["score"])

17.109785638339876


In [ ]:
def compute_bleu(src_sentences, tgt_sentences, batch_size=32):
  pred = []

  for i in tqdm(range(0, len(src_sentences), batch_size)):
    batch = src_sentences[i:i + batch_size]
    encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
    translated_output = model.generate(**encoded_input)
    translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_output]
    pred.extend(translated_batch)

  res = metric.compute(predictions=pred, references=tgt_sentences)
  return res["score"]

In [ ]:
def get_translation_opus(test_string, model, tokenizer):
  tokens = tokenizer(test_string, return_tensors="pt", padding=False).to("cuda")
  translated = model.generate(**tokens)
  return str([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])


def get_translation_nllb(test_string, model, tokenizer):
  inputs = tokenizer(test_string, return_tensors="pt").to("cuda")
  translated_tokens = model.generate(
      **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
  )
  return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]



In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

opus_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
opus_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en").to("cuda")
opus_ft_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Cantonese-NLP/model/opus-mt-zh-en-finetuned", local_files_only=True).to("cuda")


nllb_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
nllb_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to("cuda")
nllb_ft_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Cantonese-NLP/model/nllb-200-distilled-600M-finetuned", local_files_only=True).to("cuda")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def get_result(test_string, gold_standard, opus_tokenizer, opus_model, opus_ft_model, nllb_tokenizer, nllb_model, nllb_ft_model):
  print(f"Original Text: {test_string}")
  print(f"Gold Standard Translation: {gold_standard}")
  print(f"--------------------------------------------")
  print(f"OPUS-MT-en-zh Translation: {get_translation_opus(test_string, opus_model, opus_tokenizer)}")
  print(f"OPUS-MT-en-zh-finetuned Translation: {get_translation_opus(test_string, opus_ft_model, opus_tokenizer)}")
  print(f"--------------------------------------------")
  print(f"NLLB-200 Translation: {get_translation_nllb(test_string, nllb_model, nllb_tokenizer)}")
  print(f"NLLB-200-finetuned Translation: {get_translation_nllb(test_string, nllb_ft_model, nllb_tokenizer)}")

In [ ]:
import random
n = random.randint(0, len(dataset["test"]) - 1)
test = dataset["test"][n]

get_result(test_string=test["zh"],
           gold_standard=test["en"],
           opus_tokenizer=opus_tokenizer,
           opus_model=opus_model,
           opus_ft_model=opus_ft_model,
           nllb_tokenizer=nllb_tokenizer,
           nllb_model=nllb_model,
           nllb_ft_model=nllb_ft_model)

Original Text: 落咗髮泥之後，你要用手摷鬆個頭。
Gold Standard Translation: After applying hair wax, you have to rake your hand through your hair.
--------------------------------------------
OPUS-MT-en-zh Translation: After the hair's mud, you have to relax your head with your hands.
OPUS-MT-en-zh-finetuned Translation: You need to loosen your hair with your hands after you've got a muddled hair.
--------------------------------------------
NLLB-200 Translation: After you lose your hair, you'll have to use your hands to loosen your head.
NLLB-200-finetuned Translation: After you lose your hair, you have to apply a hair dressing hand.
